In [57]:
from gensim.summarization.summarizer import summarize
from gensim.models import Word2Vec, TfidfModel
from gensim.corpora import Dictionary
from gensim.matutils import softcossim

from TextProcessing import lemmatize_word
from database import GetArticles
from random import randrange
from helpers import flatten
from tqdm import tqdm

import pickle
import spacy
import os
import re

In [58]:
nlp = spacy.load('de')
word2vec = Word2Vec.load('./data/noun-chunks/w2v.bin')
dictionary = Dictionary.load('./data/noun-chunks/dict.bin')
tfidf = TfidfModel.load('./data/noun-chunks/tfidf.bin')

In [59]:
similarity_matrix_file = './data/noun-chunks/similarity-matrix.pkl'
if os.path.isfile(similarity_matrix_file) and os.path.getsize(similarity_matrix_file) > 0:
    with open(similarity_matrix_file, 'rb') as f:
        similarity_matrix = pickle.load(f)
else:
    similarity_matrix = word2vec.wv.similarity_matrix(dictionary, tfidf)
    with open(similarity_matrix_file, 'wb') as f:
        pickle.dump(similarity_matrix, f)

In [60]:
def preprocess(text):
        # Clean text and remove double spacy and special chars and copyrights.
        text = re.sub(' +', ' ', text)
        text = re.sub(r'[^\w äöüÄÖÜß\"\:\,\.?\!\-\_ ]', '', text)
        text = re.sub(' 0 0 ', '', text)
        text = re.sub('Mehr zum Thema\:(.*)', '', text)
        text = re.sub('Berliner Morgenpost [0-9]{4} (.*) Alle Rechte vorbehalten\.', '', text)  # removes copyright if 'Mehr zum Thema' is not in text
        text = re.sub('&', 'und', text)
        text = re.sub(' +', ' ', text)
        text = re.sub(' \.', '.', text)
        text = re.sub('Berliner Morgenpost [0-9]{4} Alle Rechte vorbehalten\.', '', text)
        text = text.strip()

        
        doc = nlp(text)

        sentences = []
        for sentence in doc.sents:  # Split text into sentences
            chunks = []

            for chunk in sentence.noun_chunks:  # Get noun_chunks out of sentence
                tokens = chunk.text.lower().split(' ')
                tokens = list(filter(lambda token: not nlp.vocab[token].is_stop, tokens)) # Remove stopwords
                tokens = list(filter(lambda token: token != '', tokens)) # Remove empty tokens
                tokens = list(map(lambda token: lemmatize_word(token), tokens))  # Lemmatize

                chunk = ' '.join(tokens) # join stopword cleaned noun-chunks together again
                chunk = chunk.lower()  # lower all the text

                if chunk != '':
                    chunks.append(chunk)

            if len(chunks) > 0:  # Only append non empty tokens
                sentences.append(chunks)

        return flatten(sentences)  # Merge words of the sentences together to a document array

In [61]:
def similarity(text1, text2):
    preprocessed_text1 = preprocess(text1)
    preprocessed_text2 = preprocess(text2)

    bow1 = dictionary.doc2bow(preprocessed_text1)
    bow2 = dictionary.doc2bow(preprocessed_text2)
    
    return softcossim(bow1, bow2, similarity_matrix)

In [62]:
text = '''
    Die Berliner Schulen kommen nicht zur Ruhe. „Wir brauchen dringend Unterstützung!“, ist ein Brandbrief der Mildred-Harnack-Schule in Lichtenberg überschrieben, einer Inte­grierten Sekundarschule mit gymnasialer Oberstufe. Darin wird geklagt, dass Schüler mit Förderbedarf einfach auf die Regelklassen verteilt würden, ohne dass sie die meiste Zeit einen Förderlehrer zur Seite gestellt bekämen. Ein Unterricht sei so oft nur schwer möglich. Außerdem seien die Klassen zu groß, gerade nach der 9. Klasse. „Wir sind mehr als am Limit!!!“, endet der Brief des Kollegiums.

    Dieser Brandbrief ist der letzte einer ganzen Reihe aus verschiedenen Bezirken, die von Schul-Kollegien oder Erziehern verfasst wurden. Immer geht es darin um die Überforderung, da die Schulverwaltung einerseits die Inklusion – also die Integration aller – möchte, aber nicht das nötige Personal dafür stellt. Dazu kommt, dass die Schulen in Lichtenberg aus allen Nähten platzen. An den Berliner Grundschulen sollen aktuell 1129 Plätze fehlen, auch in den weiterführenden Schulen wird es eng. Grund ist der starke Zuzug von Familien. Seit Montag halten Eltern deshalb eine Mahnwache unter dem Motto „Hilfe, wir platzen!“ vor dem Rathaus Lichtenberg ab.

    „Wir stehen mit dem Rücken zur Wand“
    Doris Siebernik, Vorsitzende der GEW Berlin, sprach angesichts der Lage von einem „Flächenbrand“ in der Stadt. Bildungssenatorin Sandra Scheeres (SPD) müsse jetzt dringend die richtigen Signale setzen. Die Kollegen in den Schulen müssten entlastet werden. „Wir stehen mit dem Rücken zur Wand.“

    Die GEW Berlin präsentierte am Dienstag Ideen, wie man dem Fachkräftemangel an den Schulen entgegenwirken könne – eine Reaktion auf die Vorschläge von Scheeres Anfang des Monats. So will man verstärkt Verwaltungs- und Gesundheitspersonal in den Schulen einstellen, um Lehrer und Direktoren zu entlasten. Deutlich kritisiert die GEW den Umgang der Senatsschulverwaltung mit Quereinsteigern. Die müssten besser vorbereitet und entlastet werden. Auch müsse der Weg des Quereinstiegs transparenter werden.

    Klar ist, es werden zu Schulanfang rund 500 Lehrer fehlen. Deshalb müsse man über die Liste der Mangelfächer neu nachdenken. „Wenn die Lücke so riesig ist, muss man weiterdenken“, so GEW-Vorstand Udo Mertens. Als „zynisch“ empfindet man das Argument der Senatsschulverwaltung, die Stundentafel sei mit 140 Prozent übererfüllt. „Das ist kein Bonus“, so Tom Erdmann von der GEW. Die 40 Prozent mehr seien notwendig, um die Inklusion umsetzen zu können. „Was hilft der zusätzliche Sonderpädagoge, wenn er als Vertretungslehrer die Nebenklasse übernimmt.“
'''

In [63]:
print(summarize(text, word_count=40))

„Wir brauchen dringend Unterstützung!“, ist ein Brandbrief der Mildred-Harnack-Schule in Lichtenberg überschrieben, einer Inte­grierten Sekundarschule mit gymnasialer Oberstufe.
Die GEW Berlin präsentierte am Dienstag Ideen, wie man dem Fachkräftemangel an den Schulen entgegenwirken könne – eine Reaktion auf die Vorschläge von Scheeres Anfang des Monats.


In [64]:
def build_graph(text):
    doc = nlp(text)
    sents = [sent.text.strip() for sent in doc.sents]
    graph = []
    
    for i, sentI in enumerate(sents):
        for o, sentO in enumerate(sents):
            if i is not o:
                graph.append((i, o, sentI, sentO, similarity(sentI, sentO)))
    return graph

In [65]:
print(build_graph(text))

[(0, 1, 'Die Berliner Schulen kommen nicht zur Ruhe.', '„Wir brauchen dringend Unterstützung!', 0.0), (0, 2, 'Die Berliner Schulen kommen nicht zur Ruhe.', '“, ist ein Brandbrief der Mildred-Harnack-Schule in Lichtenberg überschrieben, einer Inte\xadgrierten Sekundarschule mit gymnasialer Oberstufe.', 0.0), (0, 3, 'Die Berliner Schulen kommen nicht zur Ruhe.', 'Darin wird geklagt, dass Schüler mit Förderbedarf einfach auf die Regelklassen verteilt würden, ohne dass sie die meiste Zeit einen Förderlehrer zur Seite gestellt bekämen.', 0.0), (0, 4, 'Die Berliner Schulen kommen nicht zur Ruhe.', 'Ein Unterricht sei so oft nur schwer möglich.', 0.0), (0, 5, 'Die Berliner Schulen kommen nicht zur Ruhe.', 'Außerdem seien die Klassen zu groß, gerade nach der 9. Klasse.', 0.0), (0, 6, 'Die Berliner Schulen kommen nicht zur Ruhe.', '„Wir sind mehr als am Limit!!', 0.0), (0, 7, 'Die Berliner Schulen kommen nicht zur Ruhe.', '!“, endet der Brief des Kollegiums.', 0.0), (0, 8, 'Die Berliner Schulen